In [1]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!ls "/content/drive/My Drive/evodata"

X1.h5  Y1.h5


In [0]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [0]:
!cp "/content/drive/My Drive/evodata/Y1.h5" data/

In [0]:
!cp "/content/drive/My Drive/evodata/X1.h5" data/

In [0]:
!ls data

X1.h5  Y1.h5


In [0]:
!chmod 777 data/X1.h5

In [0]:
!chmod 777 data/Y1.h5

In [2]:
# import numpy as np
# np.__version__
!pip install scikit-multilearn
!pip install numpy==1.15.4

     |████████████████████████████████| 92kB 4.2MB/s 


     |████████████████████████████████| 92kB 4.2MB/s 


In [3]:
import pandas as pd
import numpy as np
from torch.autograd import Variable
from skmultilearn.problem_transform import LabelPowerset
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
# from skmultilearn.problem_transform import LabelPowerset

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
go_terms_to_classify = ['go_9401', 'go_45454']

In [0]:
with pd.HDFStore('/content/drive/My Drive/evodata/Y1.h5',  mode='r') as store:
    Y = store.select('Y')

with pd.HDFStore('/content/drive/My Drive/evodata/X1.h5',  mode='r') as store:
    X = store.select('X')

In [0]:
X['InclusionBodiesProbPos'] = (X['InclusionBodiesProbability']>=0)*X['InclusionBodiesProbability']
X['InclusionBodiesProbNeg'] = (X['InclusionBodiesProbability']<0)*X['InclusionBodiesProbability']


def location_type_numeric(x1):
    if x1 == '0':
        return 1
    elif x1 == 'SignalP_gram_plus_S':
        return 2
    elif x1 == 'SignalP_gram_minus_S':
        return 3
    elif x1 == 'T4SEpre_bpbAac_T4SE':
        return 4
    elif x1 == 'hmmsearch_Tat':
        return 5
    elif x1 == 'T3_MM_T3SS':
        return 6
    elif x1 == 'SignalP_gram_plus_S':
        return 7
    elif x1 == 'hmmsearch_T6S':
        return 8
    elif x1 == 'hmmsearch_T1S':
        return 9
    else:
        raise Exception(x1)

X['loc_type_numeric'] = X['loc_type'].apply(location_type_numeric)




In [0]:
Y.sort_values('cluster', inplace=True)
X.sort_values('cluster_is', inplace=True)

Y.drop('cluster', inplace=True,axis=1)
X.drop(['cluster_is', 'InclusionBodiesProbability', 'loc_type' ], inplace=True,axis=1)



In [0]:
labels_on_sums = Y.sum(axis=0)
labels_on_sums.sort_values(ascending=False, inplace=True)

In [0]:
go_terms_to_classify = list(labels_on_sums[0:10].keys())

In [0]:
Y = Y[go_terms_to_classify] if go_terms_to_classify is not None else Y

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [0]:
# lp = LabelPowerset()
# yt = lp.transform(y_train)
# rus = RandomOverSampler(random_state=42)
# X_res, y_res = rus.fit_resample(X_train.values, yt)

X_res = X_train.values
y_res = y_train.values


In [17]:
# y_res = lp.inverse_transform(y_res)
# y_res = y_res.todense()
# y_res =np.array(y_res)


(1770190, 10)

In [0]:

class DatasetLoader(Dataset):

    def __init__(self, x_df, y_df):
        self.x_df = x_df
        self.y_df = y_df


    def __len__(self):
        return len(self.x_df)

    def __getitem__(self, idx):
        x = self.x_df[idx]
        #y = self.y_df.iloc[idx]['go_9401']
        y = self.y_df[idx]

        return x.reshape(-1), y.reshape(-1)

        # return np.asarray(x.tolist()).reshape(-1), np.asarray([y]).reshape(-1)


train_dataset_loader = DatasetLoader(X_res, y_res)
train_loader = DataLoader(train_dataset_loader, batch_size=100, shuffle=True, num_workers=0)


test_dataset_loader = DatasetLoader(X_test.values, y_test.values)
test_loader = DataLoader(test_dataset_loader, batch_size=100, shuffle=True, num_workers=0)


class GoTermNN(nn.Module):
    def __init__(self, num_classes, num_inputs=62,  dropout_prob=0.3):
        super(GoTermNN, self).__init__()
        self.pipe = nn.Sequential(
            nn.Linear(num_inputs, 50),
            nn.ReLU(),

            nn.Linear(50, 20),
            nn.ReLU(True),

            nn.Linear(20, num_classes)
        )


    def forward(self, x):
        return self.pipe(x)





In [0]:

def train(model, device, train_loader, optimizer, epoch, writer, criterion):
    model.train()
    running_loss = 0.0
    running_correct = 0.0
    calc_count = 0.0
    labels_list = []
    scores_list = []
    loss1 = 0.1
    for x, (images, labels) in enumerate(train_loader):
        images = images.float().to(device)
        labels = labels.float().to(device)
        optimizer.zero_grad()

        images = Variable(images)
        labels = Variable(labels)

        output = model(images)
        calc_count += output.data.shape[0]
        _, pred = torch.max(output.data, 1)
        loss = criterion(output, labels)

        #out1 = F.sigmoid(output)
        out1 = torch.sigmoid(output)
        
        labels_list.extend(labels.cpu().numpy().tolist())
        scores_list.extend(out1.data.tolist())

        current_loss = loss.item()
        running_loss += current_loss

        loss1 += current_loss

        if x > 0 and  x %10000 == 0:
            
            
            a1 = np.asarray(scores_list)
#             a1[a1 < 0.5] = 0
#             a1[a1 >= 0.5] = 1

            #f1_res = f1_score(a1.astype(np.int32), np.asarray(labels_list), average='weighted')
            
            f1_res_w =  f1_score(np.asarray(labels_list), a1 > 0.5, average="weighted")

            
            print('f1_res_w {0}'.format(f1_res_w))
            
#             f1_res_s=  f1_score(np.asarray(labels_list), a1 > 0.5, average="samples")

            
#             print('f1_res_s {0}'.format(f1_res_s))
            
            auc = roc_auc_score(np.asarray(labels_list, dtype='float32'), np.asarray(scores_list, dtype='float32'))
            
            #f1_res = f1_score(np.asarray(labels_list, dtype='float32'), np.asarray(scores_list, dtype='float32'))
            #print('f1 {0}'.format(f1_res))
            print('auc {0}'.format(auc))
            print('loss1 {0}'.format(loss1))

            labels_list = []
            scores_list = []
            
            loss1 = 0.0


        #writer.add_scalar('train_loss', loss.item())

        loss.backward()
        optimizer.step()

    accuracy = round(running_correct / calc_count, 4)
    #writer.add_scalar('train_accuracy', accuracy)
    return round(running_loss / calc_count, 4), accuracy


def test(model, device, test_loader, epoch, writer, criterion):
    model.eval()
    running_correct = 0.0
    running_loss = 0.0
    calc_count = 0.0
    labels_list = []
    scores_list = []
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.float().to(device)
            labels = labels.float().to(device)

            images = Variable(images)
            labels = Variable(labels)
            outputs = model(images)
            loss = criterion(outputs, labels)
            calc_count += outputs.data.shape[0]
            
            running_loss += loss.item()

            #out1 = F.sigmoid(outputs)
            
            out1 = torch.sigmoid(outputs)
            labels_list.extend(labels.cpu().numpy().tolist())
            scores_list.extend(out1.data.tolist())

    auc = roc_auc_score(np.asarray(labels_list, dtype='float32'), np.asarray(scores_list, dtype='float32'))
    
    
    a1 = np.asarray(scores_list)
#     a1[a1 <= 0.5] = 0
#     a1[a1 > 0.5] = 1

    #f1_res = f1_score(a1.astype(np.int32), np.asarray(labels_list), average='weighted')

    #print('f1 {0}'.format(f1_res))
    
    f1_res_w =  f1_score(np.asarray(labels_list), a1 > 0.5, average="weighted")

            
    print('test f1_res_w {0}'.format(f1_res_w))
            
#     f1_res_s=  f1_score(np.asarray(labels_list), a1 > 0.5, average="samples")

#     print('test f1_res_s {0}'.format(f1_res_s))
            
            
    
    #f1_res = f1_score(np.asarray(labels_list, dtype='float32'), np.asarray(scores_list, dtype='float32'))
    #print('test f1 {0}'.format(f1_res))
      
    print('test auc {0}'.format(auc))
    accuracy = round(running_correct / calc_count, 4)

    return round(running_loss / calc_count, 4), accuracy

In [0]:


from datetime import datetime

use_cuda = torch.cuda.is_available()

torch.manual_seed(999)

if use_cuda:
    torch.cuda.manual_seed(999)
    torch.cuda.manual_seed_all(999)

print('use_cuda = {}'.format(use_cuda))
device = torch.device("cuda" if use_cuda else "cpu")


criterion = torch.nn.BCEWithLogitsLoss().to(device)

torch.manual_seed(999)
now = datetime.now()

model = GoTermNN(y_res.shape[1]).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(20):
    print('epoch={}'.format(epoch))
    train_loss, train_correct = train(model, device, train_loader, optimizer, epoch, None, criterion)
    print('train {0} - {1}'.format(train_loss, train_correct))
    test_loss, test_correct = test(model, device, test_loader, epoch, None, criterion)

    print('test {0} - {1}'.format(test_loss, test_correct))



print(1)



use_cuda = True
epoch=0


In [0]:
y_true = np.array([[1,0,0,0], [1,1,0,0], [1,1,1,1]])
y_pred = np.array([[1,0,0,0], [1,1,1,0], [1,1,1,1]])

f1_score(y_true=y_true, y_pred=y_pred, average='weighted')

0.9523809523809524

use_cuda = False
